In [1]:
from manim import *
from manim_extensions import *
from stickman import *
import itertools as it
from global_funcs import *

Manim Community v0.15.2

In [20]:
%%manim -v WARNING -qm GravitationalGround

class GravitationalGround(Scene):
    def construct(self):
        skyscraper = SVGMobject("./svgs/skyscraper.svg").scale(3)
        stickman = StickMan(leg_angle=np.pi / 6).scale(1/10).next_to(skyscraper.get_top(), UP, buff=0.05).align_to(skyscraper, RIGHT)
        ball = Circle(radius=0.03, color=RED, fill_opacity=1).move_to(stickman.get_right_arm_end() + RIGHT * 0.03)
        self.add(skyscraper, stickman, ball)

        tracker_width = 0.3
        line1 = Line(ORIGIN, RIGHT * tracker_width).next_to(skyscraper, RIGHT).align_to(skyscraper, DOWN)
        line2 = line1.copy()
        line3 = always_redraw(lambda: DashedLine(line1.get_center(), line2.get_center()))
        
        self.play(
            *[FadeIn(line) for line in (line1, line2, line3)]
        )
        self.play(line2.animate.next_to(ball, RIGHT).align_to(line1, LEFT), run_time=3)
        # line3.clear_updaters()
        self.wait()
        h_label = Tex("$h=$ tall").next_to(line3, RIGHT)
        self.play(FadeIn(h_label))
        self.wait()

        ground_label = Text("Ground").scale(0.6).shift(RIGHT * 4 + DOWN * 2).set_color(GRAY)
        ground_arrow = Arrow(ground_label.get_critical_point(DL), line1.get_right()).set_color(GRAY)
        self.play(
            *[FadeIn(mob) for mob in (ground_label, ground_arrow)]
        )
        self.wait()

        self.play(
            *[FadeOut(mob) for mob in (line1, line2, line3, h_label, ground_label, ground_arrow)]
        )
        self.play(
            VGroup(skyscraper, stickman, ball).animate.scale(scale_factor=10, about_point=skyscraper.get_critical_point(UR)).shift(DOWN * 3)
        )
        self.play(stickman.animate.shift(LEFT), ball.animate.shift(LEFT))
        self.wait()

        line1.next_to(skyscraper, RIGHT).align_to(skyscraper, UP)
        line2.next_to(ball, RIGHT).align_to(line1, LEFT)
        line3.update()
        h_label = Tex("$h=$ short").next_to(line3, RIGHT)

        self.play(*[FadeIn(mob) for mob in (line1, line2, line3)])

        ground_label = Text("Ground").scale(0.6).shift(RIGHT * 4).set_color(GRAY)
        ground_arrow = Arrow(ground_label.get_critical_point(LEFT), line1.get_right()).set_color(GRAY)
        self.play(*[FadeIn(mob) for mob in (ground_label, ground_arrow)])

        self.wait()

        self.play(FadeIn(h_label))

        self.wait()



    

In [2]:
%%manim -v WARNING -ql ZeroElectricPotentialEnergy

class ZeroElectricPotentialEnergy(Scene):
    def construct(self):
        ax = Axes(
            (0, 14, 1),
            (0, 1200, 100),
            x_length=14,
            y_length=4,
            x_axis_config={
                "include_numbers": True
            },
            y_axis_config={
                "include_numbers": True,
                "numbers_to_exclude": [1100]
            }
        ).align_on_border(UL)
        self.add(ax)

        pos_tracker = ValueTracker(1)
        k = 250
        q1 = q2 = 1
        def force():
            r = pos_tracker.get_value()
            return 4 * q1 * q2 / r ** 2
        def get_pe(r=None):
            if r is None:
                r = pos_tracker.get_value()
            return k * q1 * q2 / r

        graph = ax.plot(get_pe, x_range=(0.15, 14)).set_color(ELECTRIC_PE_COLOR)
        dot = always_redraw(lambda: Dot(ax.coords_to_point(pos_tracker.get_value(), get_pe())).set_color(ELECTRIC_PE_COLOR))
        self.add(graph, dot)

        e1 = Circle(radius=0.3, color=YELLOW, fill_opacity=1).align_to(ax.coords_to_point(0, 0) + LEFT * 0.3, LEFT).shift(DOWN * 2)
        e1_arrow = always_redraw(lambda: Arrow(ORIGIN, LEFT * force(), buff=0).next_to(e1, LEFT, buff=0).set_color(ELECTRIC_PE_COLOR))
        e2 = always_redraw(lambda: e1.copy().shift(RIGHT * pos_tracker.get_value()))
        e2_arrow = always_redraw(lambda: Arrow(ORIGIN, RIGHT * force(), buff=0).next_to(e2, RIGHT, buff=0).set_color(ELECTRIC_PE_COLOR))
        self.add(e1, e1_arrow, e2, e2_arrow)

        pe = always_redraw(lambda: MathTex(f"PE={round(get_pe(), 3)}J").shift(UP))
        self.add(pe)

        tracker_size = 0.3
        line1 = Line(ORIGIN, DOWN * tracker_size).next_to(e1, DOWN)
        line2 = always_redraw(lambda: line1.copy().next_to(e2, DOWN))
        line3 = always_redraw(lambda: DashedLine(line1.get_center(), line2.get_center()))
        def r_label_pos_updater(m: Mobject):
            m.next_to(line3, DOWN)
            if m.get_left()[0] > 0:
                m.align_to(ORIGIN, LEFT)
        r_label = always_redraw(lambda: MathTex(f"r={round(pos_tracker.get_value(), 1)}m"))
        r_label.add_updater(r_label_pos_updater)
        self.add(line1, line2, line3, r_label)

        self.play(pos_tracker.animate.set_value(1000), run_time=4, rate_func=linear)
        
        pe.clear_updaters()
        pe.become(MathTex(f"PE=0J").shift(UP))
        r_label.become(MathTex(f"r=\infty m").move_to(r_label))
        # r_label.clear_updaters()
        self.remove(dot)
        self.wait()



In [61]:
class ElectronBox(VGroup):
    def __init__(
        self,
        width,
        height,
        electron_generator=None,
        electron_position_initializer=None,
        post_electron_modifier=None,
        chunk_density=(10, 10),
        num_rows=10,
        num_cols=10,
        draw_force_lines=False,
        draw_chunk_lines=False,
        *args,
        **kwargs
    ):
        # self.chunk_density = (rows, cols)
        super().__init__()
        self._box = Rectangle(width=width, height=height, *args, **kwargs)
        if electron_generator is None:
            electron_generator = self.default_electron_generator
        if electron_position_initializer is None:
            electron_position_initializer = self.initialize_electron_position
        if post_electron_modifier is None:
            post_electron_modifier = lambda electrons: electrons
        self.draw_force_lines = draw_force_lines
        self.chunk_density = chunk_density
        self._electrons = [electron_position_initializer(self, electron_generator(), r, c, num_rows, num_cols) for c in range(num_cols) for r in range(num_rows)]
        new_electrons = post_electron_modifier(self._electrons)
        self.add(*new_electrons)
        self.add(self._box)
        self.add_updater(self.update_electron_positions)

        if draw_chunk_lines:
            ul = self._box.get_critical_point(UL)
            dl = self._box.get_critical_point(DL)
            ur = self._box.get_critical_point(UR)
            dr = self._box.get_critical_point(DR)
            gridx = VGroup(*[Line(ul, dl).shift(RIGHT * self._box.width  * i / chunk_density[0]) for i in range(chunk_density[0])]).set_color(GRAY)
            gridy = VGroup(*[Line(ul, ur).shift(DOWN  * self._box.height * i / chunk_density[1]) for i in range(chunk_density[1])]).set_color(GRAY)
            self.add(gridx, gridy)

    def update_electron_positions(self, m, dt):
        # Assign each electron to a chunk
        chunks = [[[] for _ in range(self.chunk_density[0])] for _ in range(self.chunk_density[1])]
        chunk_width = self._box.width / self.chunk_density[1]
        chunk_height = self._box.height / self.chunk_density[0]
        zero = self._box.get_critical_point(UL)
        end = self._box.get_critical_point(DR)
        for electron in self._electrons:
            electron: Mobject
            center = electron[0].get_center() - zero
            c = int(center[0] / chunk_width)
            r = int(-center[1] / chunk_height)
            if c < 0:
                c = 0
            elif c >= self.chunk_density[1]:
                c = self.chunk_density[1] - 1
            if r < 0:
                r = 0
            elif r >= self.chunk_density[0]:
                r = self.chunk_density[0] - 1
            chunks[r][c].append(electron)
        k = 0.1
        wall_k = 2
        speed_damping = -0.04
        speed_multiplier = np.exp(speed_damping * dt)
        get_f = lambda e1, e2: k / np.dot(e1.get_center() - e2.get_center(), e1.get_center() - e2.get_center())
        get_f_lw = lambda e: wall_k / (e.get_center()[0] - zero[0]) ** 2
        get_f_uw = lambda e: wall_k / (e.get_center()[1] - zero[1]) ** 2
        get_f_rw = lambda e: wall_k / (e.get_center()[0] - end[0]) ** 2
        get_f_bw = lambda e: wall_k / (e.get_center()[1] - end[1]) ** 2
        for r in range(self.chunk_density[0]):
            for c in range(self.chunk_density[1]):
                for e1 in chunks[r][c]:
                    f = np.zeros(3)
                    for e2 in it.chain(*(
                        electron
                        for row in 
                        chunks[max(r - 1, 0):min(r + 2, self.chunk_density[0])]
                        for electron in
                        row[max(c - 1, 0):min(c + 2, self.chunk_density[1])] 
                    )):
                        if e1 is e2:
                            continue
                        f += get_f(e1, e2) * norm(e1.get_center() - e2.get_center())
                    if r == 0:
                        f += get_f_uw(e1) * DOWN
                    elif r == self.chunk_density[0] - 1:
                        f += get_f_bw(e1) * UP
                    if c == 0:
                        f += get_f_lw(e1) * RIGHT
                    elif c == self.chunk_density[1] - 1:
                        f += get_f_rw(e1) * LEFT
                    f_mag = np.sqrt(np.dot(f, f))
                    if f_mag >= 1:
                        f *= 1 / f_mag
                    if np.any(np.isnan(f)):
                        f = np.zeros(3)
                    e1.f = f
        for electron in self._electrons:
            electron.shift(electron.v * dt)
            electron.v += electron.f * dt
            electron.v *= speed_multiplier
            center = electron[0].get_center()
            proper_pos = np.array([
                max(zero[0], min(center[0], end[0])),
                max(end[1], min(center[1], zero[1])),
                0
            ])
            if proper_pos[0] != center[0]:
                electron.v[0] *= -0.1
            if proper_pos[1] != center[1]:
                electron.v[1] *= -0.1
            electron.shift(proper_pos - center)
                    



    def default_electron_generator(self):
        group = VGroup(
            Circle(radius=0.1, fill_opacity=1, color=YELLOW),
        )
        if self.draw_force_lines:
            group += Line()
            group.add_updater(lambda m: m[1].become(Line(ORIGIN, m.f).shift(m[0].get_center())))
        return group

    def initialize_electron_position(self, box, electron: Mobject, r, c, num_rows, num_cols):
        # Move the electron to the right place and
        # give it a velocity of zero
        w, h = self._box.width, self._box.height
        x0, y0, _ = self._box.get_critical_point(UL)
        x, y = w / (num_cols + 1) * (c + 1) + x0, -h / (num_rows + 1) * (r + 1) + y0
        electron.move_to(np.array([x, y, 0]))
        electron.v = np.zeros(3)
        return electron


In [64]:
%%manim -v WARNING -qh ElectronBoxTest

class ElectronBoxTest(Scene):
    def construct(self):
        def initialize_electron_position(box, electron: Mobject, r, c, num_rows, num_cols):
            # Move the electron to the right place and
            # give it a velocity of zero
            c /= 2
            w, h = box._box.width, box._box.height
            x0, y0, _ = box._box.get_critical_point(UL)
            x, y = w / (num_cols + 1) * (c + 1) + x0, -h / (num_rows + 1) * (r + 1) + y0
            electron.move_to(np.array([x, y, 0]))
            electron.v = np.zeros(3)
            return electron
        def electron_generator():
            electron = VGroup()
            circle = Circle(radius=0.1, color=YELLOW, fill_opacity=1)
            voltage = Circle(radius=2).set_fill((RED, BLACK, BLACK), opacity=(1, 0, 0)).set_stroke(BLACK, opacity=0)
            voltage.radial_gradient = True
            electron += voltage
            electron += circle
            return electron
        def post_electron_modifier(electrons: VGroup):
            new_electrons = list(it.chain(*electrons))
            new_electrons.sort(key=lambda x: x.radius, reverse=True)
            return new_electrons
        box = ElectronBox(
            4, 4, chunk_density=(10, 10), num_rows=10, num_cols=10,
            electron_position_initializer=initialize_electron_position, electron_generator=electron_generator,
            post_electron_modifier=post_electron_modifier
        )
        self.add(box)
        self.wait(10)
